In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

# To install fairseq from source and develop locally:
!git clone https://github.com/pytorch/fairseq
%cd /content/fairseq
!pip install --editable .

# Download and prepare the data
%cd /content/fairseq
%cd examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

# Preprocess/binarize the data
TEXT="examples/translation/iwslt14.tokenized.de-en"
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref "examples/translation/iwslt14.tokenized.de-en/valid" --testpref "examples/translation/iwslt14.tokenized.de-en/test" \
    --destdir data-bin/iwslt14.tokenized.de-en \
    --workers 20

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Cloning into 'fairseq'...
remote: Enumerating objects: 11889, done.
remote: Total 11889 (delta 0), reused 0 (delta 0), pack-reused 11889
Receiving objects: 100% (11889/11889), 5.55 MiB | 4.51 MiB/s, done.
Resolving deltas: 100% (8850/8850), done.
/content/fairseq
Obtaining file:///content/fairseq
     |████████████████████████████████| 645kB 6.4MB/s 
  Running setup.py develop for fairseq
/content/fairseq
/content/fairseq/examples/translation
Cloning Moses gi

In [0]:

SAVE="/content/gdrive/My Drive/colab/payless/save/dynamic_conv_iwslt"

# Training
%cd /content/fairseq
!mkdir -p "{SAVE}" 
!CUDA_VISIBLE_DEVICES=0 $(which fairseq-train) data-bin/iwslt14.tokenized.de-en \
    --clip-norm 0 --optimizer adam --lr 0.0005 \
    --source-lang de --target-lang en --max-tokens 4000 --no-progress-bar \
    --log-interval 100 --min-lr '1e-09' --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --lr-scheduler inverse_sqrt \
    --ddp-backend=no_c10d \
    --max-update 50000 --warmup-updates 4000 --warmup-init-lr '1e-07' \
    --adam-betas '(0.9, 0.98)' --keep-last-epochs 10 \
    -a lightconv_iwslt_de_en --save-dir "{SAVE}" \
    --dropout 0.3 --attention-dropout 0.1 --weight-dropout 0.1 \
    --encoder-glu 0 --decoder-glu 0
!python scripts/average_checkpoints.py --inputs "{SAVE}" \
    --num-epoch-checkpoints 10 --output "{SAVE}/checkpoint_last10_avg.pt"

# Evaluation
!CUDA_VISIBLE_DEVICES=0 fairseq-generate data-bin/iwslt14.tokenized.de-en --path "{SAVE}/checkpoint_last10_avg.pt" --batch-size 128 --beam 4 --remove-bpe --lenpen 1 --gen-subset test --quiet 

/content/fairseq
Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='lightconv_iwslt_de_en', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data-bin/iwslt14.tokenized.de-en', dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=4, decoder_conv_dim=512, decoder_conv_type='dynamic', decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_glu=False, decoder_input_dim=512, decoder_kernel_size_list=[3, 7, 15, 31, 31, 31], decoder_layers=6, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.3, empty_cache_freq=0, encoder_attention_heads=4